# CityBikes

Send a request to CityBikes for the city of your choice. 

In [1]:
# We start by importing all the necessary libraries
import requests
import json
import pandas as pd
import import_ipynb

In [2]:
# First we find CityBikes API endpoint for Toronto and use it for URL
# It is /bixi-toronto
url = "http://api.citybik.es/v2/networks/bixi-toronto"

# Then we send the request to the CityBikes API
response = requests.get(url)


In [3]:
response = requests.get(url)
#Then we check the status code to ensure the request was successful
response

<Response [200]>

In [4]:
# After that, we will parse the response as JSON
data = response.json()

In [5]:
# print(json.dumps(data))
# We can print data to check it

In [6]:
df = pd.DataFrame(data)
df.head()

,network
id,bixi-toronto
name,Bike Share Toronto
location,"{'latitude': 43.653226, 'longitude': -79.38318..."
href,/v2/networks/bixi-toronto
company,"[Motivate International, Inc., PBSC Urban Solu..."


Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [7]:
# Extract the stations data from stations in network
stations = data['network']['stations']
    
# Prepare a list to store station information
station_info = []

# Loop through each station to get the required details
for station in stations:
    total_bikes = station['free_bikes'] + station['empty_slots']  # Calculate total bikes
    station_details = {
        'name': station['name'],
        'latitude': station['latitude'],
        'longitude': station['longitude'],
        'free_bikes': station['free_bikes'],
        'empty_slots': station['empty_slots'],
        'total_bikes': total_bikes
    }
    # Then we will add our station details in the station info list
    station_info.append(station_details)

In [8]:
df = pd.DataFrame(stations)
df.head()

,id,name,latitude,longitude,timestamp,free_bikes,empty_slots,extra
0,009f180cf35ae1285733d98ccf058313,Summerhill Ave / Maclennan Ave,43.685924,-79.376304,2024-10-21T07:24:29.197017Z,3,8,"{'uid': '7488', 'renting': 1, 'returning': 1, ..."
1,010507feed5b8d87c40cd95933ed5654,Queen St E / Joseph Duggan Rd,43.667763,-79.308117,2024-10-21T07:24:29.198847Z,15,3,"{'uid': '7695', 'renting': 1, 'returning': 1, ..."
2,0153756b9e136b96e730aaa2f048227f,Victoria Park Ave / Danforth Ave,43.691468,-79.288619,2024-10-21T07:24:29.198082Z,3,15,"{'uid': '7611', 'renting': 1, 'returning': 1, ..."
3,019e5937c3fc120cee906770bca8fa69,Navy Wharf Crt / Bremner Blvd,43.640722,-79.391051,2024-10-21T07:24:29.192916Z,8,0,"{'uid': '7054', 'renting': 1, 'returning': 1, ..."
4,01caf8b12874b091f2fccc3818e3c72e,420 Wellington St W,43.643834,-79.396649,2024-10-21T07:24:29.193539Z,8,5,"{'uid': '7123', 'renting': 1, 'returning': 1, ..."


Put your parsed results into a DataFrame.

In [9]:
# Convert the list of station details into a DataFrame
df = pd.DataFrame(station_info)

# Display the first 5 rows of the DataFrame
df.head()

,name,latitude,longitude,free_bikes,empty_slots,total_bikes
0,Summerhill Ave / Maclennan Ave,43.685924,-79.376304,3,8,11
1,Queen St E / Joseph Duggan Rd,43.667763,-79.308117,15,3,18
2,Victoria Park Ave / Danforth Ave,43.691468,-79.288619,3,15,18
3,Navy Wharf Crt / Bremner Blvd,43.640722,-79.391051,8,0,8
4,420 Wellington St W,43.643834,-79.396649,8,5,13


In [10]:
# Visualize summary of statistics
df.describe()

,latitude,longitude,free_bikes,empty_slots,total_bikes
count,852.000000,852.000000,852.000000,852.000000,852.000000
mean,43.674017,-79.394347,7.287559,11.046948,18.334507
std,0.036710,0.066238,7.561619,7.292779,7.150856
min,43.588077,-79.568250,0.000000,0.000000,5.000000
25%,43.649516,-79.431257,1.000000,5.000000,14.000000
50%,43.664024,-79.394175,5.000000,11.000000,17.000000
75%,43.688572,-79.364703,12.000000,15.000000,20.250000
max,43.792363,-79.123184,50.000000,45.000000,63.000000


In [11]:
missing_values = df.isnull().sum()
missing_percentage = (missing_values / df.shape[0]) * 100

In [12]:
# Checking for missing values
missing_data = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage})
missing_data[missing_data['Missing Values'] > 0]

,Missing Values,Percentage


In [13]:
df_bike = df.drop(columns=['free_bikes', 'empty_slots'], errors='ignore')
# Rename a column for clarity
df_bike.rename(columns={'name': 'station_name'}, inplace=True)

In [14]:
df_bike.head()

,station_name,latitude,longitude,total_bikes
0,Summerhill Ave / Maclennan Ave,43.685924,-79.376304,11
1,Queen St E / Joseph Duggan Rd,43.667763,-79.308117,18
2,Victoria Park Ave / Danforth Ave,43.691468,-79.288619,18
3,Navy Wharf Crt / Bremner Blvd,43.640722,-79.391051,8
4,420 Wellington St W,43.643834,-79.396649,13


In [15]:
df_bike.shape

(852, 4)

In [16]:
df_bike.describe()

,latitude,longitude,total_bikes
count,852.000000,852.000000,852.000000
mean,43.674017,-79.394347,18.334507
std,0.036710,0.066238,7.150856
min,43.588077,-79.568250,5.000000
25%,43.649516,-79.431257,14.000000
50%,43.664024,-79.394175,17.000000
75%,43.688572,-79.364703,20.250000
max,43.792363,-79.123184,63.000000


In [17]:
# Saved for later usage
# df_bike.to_csv('df_bike_data.csv', index=False)
# df_bike.to_pickle('df_bike_data.pkl')